In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
from selenium import webdriver
import requests
import pandas as pd
import numpy as np
import datetime
import time
import re

In [2]:
chromedriver_dir = r'C:\Users\LG\빅데이터 실습\chromedriver_win32/chromedriver.exe'
driver = webdriver.Chrome(chromedriver_dir)

In [3]:
url = 'https://tv.naver.com/jtbc.jtbcbros/playlists'

driver.get(url)
time.sleep(3)

In [4]:
# 더보기 계속 진행
for more in np.arange(1, 17):
    xpath = """//*[@id="cds_flick"]/div/div[2]/div/div/div/div/div[3]/a"""
    driver.find_element_by_xpath(xpath).click()

In [5]:
# BeautifulSoup 실행

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
print(soup)

<html lang="ko"><head>
<title id="headTitle">
    	
    		네이버 TV
    		
    		
    	
    </title>
<meta charset="utf-8"/>
<meta content="text/javascript" http-equiv="Content-Script-Type"/>
<meta content="text/css" http-equiv="Content-Style-Type"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="noindex" name="robots"/>
<meta content="summary" name="twitter:card"/>
<meta content="100012287952710" property="fb:admins"/>
<meta content="232491033870086" property="fb:app_id"/>
<meta content="https://tv.naver.com/jtbc.jtbcbros" property="og:url"/>
<meta content="아는 형님 : 네이버TV" property="og:title"/>
<meta content="이성 상실 본능 충실 형님학교에서 벌어지는 세상의 모든 놀이 ＜아는 형님＞" property="og:description"/>
<meta content="website" property="og:type"/>
<meta content="https://phinf.pstatic.net/tvcast/20180620_190/Msn4o_1529462848847MJzoW_PNG/20180620_114243_653.png?type=round_232_232" property="og:image"/>
<link href="/resources/release/css/tvcast_20211209161204.css" rel="stylesheet" type="text/c

In [17]:
# 저장할 데이터 프레임
knowingbrothers_view = pd.DataFrame(columns=("회차", "영상 이름", "조회수", "좋아요 수"))
knowingbrothers_view

,회차,영상 이름,조회수,좋아요 수


In [12]:
# 재생 목록 이름만 가져옴
# 회차에 해당하는 리스트만 가져옴
playlist_names = []

playlists = soup.find_all('div', 'playlist _MM_CARD')
# 재생 목록 이름 리스트

tmp = -1
for playlist in playlists:
    tmp += 1
    if tmp < 4: continue
    if tmp == 58: break
    
    playlist_name = playlist.find('h3', 'play_tit').get_text() # 재생 목록 이름 가져옴
    playlist_name = playlist_name[:10].strip() # 재생 목록을 프로그램 이름 + 회차까지 그리고 공백 제거
    playlist_names.append(playlist_name)
        
print(playlist_names)

['아는 형님 306회', '아는 형님 305회', '아는 형님 304회', '아는 형님 303회', '아는 형님 302회', '아는 형님 301회', '아는 형님 300회', '아는 형님 299회', '아는 형님 298회', '아는 형님 297회', '아는 형님 296회', '아는 형님 295회', '아는 형님 294회', '아는 형님 293회', '아는 형님 292회', '아는 형님 291회', '아는 형님 290회', '아는 형님 289회', '아는 형님 288회', '아는 형님 287회', '아는 형님 286회', '아는 형님 285회', '아는 형님 284회', '아는 형님 283회', '아는 형님 282회', '아는 형님 281회', '아는 형님 280회', '아는 형님 279회', '아는 형님 278회', '아는 형님 277회', '아는 형님 276회', '아는 형님 275회', '아는 형님 274회', '아는 형님 273회', '아는 형님 272회', '아는 형님 271회', '아는 형님 270회', '아는 형님 269회', '아는 형님 268회', '아는 형님 267회', '아는 형님 266회', '아는 형님 265회', '아는 형님 264회', '아는 형님 263회', '아는 형님 262회', '아는 형님 261회', '아는 형님 260회', '아는 형님 259회', '아는 형님 258회', '아는 형님 257회', '아는 형님 256회', '아는 형님 255회', '아는 형님 254회', '아는 형님 253회']


306 ~ 253

In [19]:
len(playlist_names)

54

In [18]:
playlists = soup.find_all('div', 'playlist _MM_CARD')
# 재생 목록 이름 리스트
i = -1
index = -1
for playlist in playlists:  
    i += 1
    if i < 4: continue
    else: index += 1
    if i == 58: break
    
    
    playlist_name_list = [] # 재생 목록 이름 저장할 리스트
    vedio_name_list = [] # 영상 이름 저장할 리스트
    view_count = [] # 조회수 저장할 리스트
    like_count = [] # 좋아요 수 저장할 리스트

    vedio_names = playlist.find_all("a") # 영상 이름
    for vedio_name in vedio_names:
        vedio_name_tmp = vedio_name.select_one("dl dt a tooltip")
        if (vedio_name_tmp != None):
            vedio_name_tmp = vedio_name.select_one("dl dt a tooltip")['title'].strip()
            vedio_name_list.append(vedio_name_tmp)
    
    name_tmp = playlist_names[index]
    view_results = playlist.find_all('span', 'hit') # 조회수들 가져옴
    for view_result in view_results:
        view_result = view_result.get_text()
        view = int(view_result[4:].replace(',', '')) # 조회수 글자제외, int형 변환
        view_count.append(view) # 리스트에 조회수 저장
        playlist_name_list.append(name_tmp)
            
    like_results = playlist.find_all('span', 'like') # 좋아요 수 가져옴
    for like_result in like_results:
        like_result = like_result.get_text().strip()
        like = int(like_result[5:].replace(',', '')) # 좋아요 수 글자제외, int형 변환
        like_count.append(like) # 리스트에 좋아요 수 저장
        
    #print(playlist_names[i])
    #break
    
    # 데이터 프레임 생성
    df = pd.DataFrame([x for x in zip(playlist_name_list, vedio_name_list, view_count, like_count)],
                        columns=("회차", "영상 이름", "조회수", "좋아요 수"))
    # 앞에서 만들어 둔 데이터 프레임에 추가
    knowingbrothers_view = knowingbrothers_view.append(df)

In [20]:
knowingbrothers_view

,회차,영상 이름,조회수,좋아요 수
0,아는 형님 306회,뮤지컬 여신 옥주현💃🏻&A+급 성질머리🔥 장동민 등장 | JTBC 211113 방송,9978,55
1,아는 형님 306회,"3人3色 매력 뿜뿜🌈 형님들의 절친, 주우재&다영&테이 | JTBC 211113 방송",6661,87
2,아는 형님 306회,빛나는 친구들 두둥등장🌟 장준&설현&시우민 Welcome~! | JTBC 21111...,10730,264
3,아는 형님 306회,세상 섹시한 타로점🔮(¬◡¬)✧ 김희철x장준의 ＜U＞♬ 합동 무대 | JTBC 21...,7171,168
4,아는 형님 306회,"김영철 또 짝사랑 중,, 비 오는 날☔️ 옥주현에게 셀프 뮤비 전송 | JTBC 2...",5105,39
...,...,...,...,...
17,아는 형님 253회,처음 문제 맞혀 신난 (여자)아이들 우기 x 미연의 'LATATA' ♬ | JTBC...,31160,529
18,아는 형님 253회,(다 같이 뿜 UP👍🏻) 모모랜드 주이 x 낸시의 한풀이 '뿜뿜'♬ 무대 | JTB...,19395,124
19,아는 형님 253회,(울컥😥) 위키미키 완전체로 나올 생각에 눈물 쏟는 유정이 | JTBC 201031 방송,49688,820
20,아는 형님 253회,[선공개] (🐶소리 대결) 채경vs지수! 의도치 않은 🐕싸움의 시작⚡️,22800,357


In [21]:
# csv 파일로 저장
# 2021-11-17 11:49 저장
knowingbrothers_view.to_csv('knowingbrothers_view_name.csv', index=False, encoding="utf-8-sig")

In [22]:
save_test = pd.read_csv('knowingbrothers_view_name.csv')
save_test

,회차,영상 이름,조회수,좋아요 수
0,아는 형님 306회,뮤지컬 여신 옥주현💃🏻&A+급 성질머리🔥 장동민 등장 | JTBC 211113 방송,9978,55
1,아는 형님 306회,"3人3色 매력 뿜뿜🌈 형님들의 절친, 주우재&다영&테이 | JTBC 211113 방송",6661,87
2,아는 형님 306회,빛나는 친구들 두둥등장🌟 장준&설현&시우민 Welcome~! | JTBC 21111...,10730,264
3,아는 형님 306회,세상 섹시한 타로점🔮(¬◡¬)✧ 김희철x장준의 ＜U＞♬ 합동 무대 | JTBC 21...,7171,168
4,아는 형님 306회,"김영철 또 짝사랑 중,, 비 오는 날☔️ 옥주현에게 셀프 뮤비 전송 | JTBC 2...",5105,39
...,...,...,...,...
1271,아는 형님 253회,처음 문제 맞혀 신난 (여자)아이들 우기 x 미연의 'LATATA' ♬ | JTBC...,31160,529
1272,아는 형님 253회,(다 같이 뿜 UP👍🏻) 모모랜드 주이 x 낸시의 한풀이 '뿜뿜'♬ 무대 | JTB...,19395,124
1273,아는 형님 253회,(울컥😥) 위키미키 완전체로 나올 생각에 눈물 쏟는 유정이 | JTBC 201031 방송,49688,820
1274,아는 형님 253회,[선공개] (🐶소리 대결) 채경vs지수! 의도치 않은 🐕싸움의 시작⚡️,22800,357


In [23]:
save_test.groupby('회차').count()

,영상 이름,조회수,좋아요 수
회차,,,
아는 형님 253회,22,22,22
아는 형님 254회,23,23,23
아는 형님 255회,20,20,20
아는 형님 256회,19,19,19
아는 형님 257회,29,29,29
아는 형님 258회,21,21,21
아는 형님 259회,24,24,24
아는 형님 260회,22,22,22
아는 형님 261회,21,21,21


In [24]:
len(save_test.groupby('회차').count().index)

54